In [1]:
# http://www-inst.eecs.berkeley.edu/~cs61b/fa14/ta-materials/apps/ab_tree_practice/

class AI:
    def __init__(self, ai_piece, opp, depth,
                 game_over_fun, eval_fun, moves_fun, next_state_fun):
        self.piece = ai_piece
        self.opp = opp
        self.depth = depth
        self.game_over = game_over_fun
        self.evaluate = eval_fun
        self.possible_moves = moves_fun
        self.next_state = next_state_fun
        
    def get_move(self, board):
        return self._minmax(board=board, player=self.piece, opp=self.opp, 
                           curr=self.piece, depth=self.depth,
                           alpha=-float("inf"), beta=float("inf"))
        
    def _minmax(self, board, player, opp, curr, depth, alpha, beta):
        if self.game_over(board, player, opp) or depth < 0:
            score = self.evaluate(board, player, opp)
            return (score, None)

        moves = self.possible_moves(board, curr)
    
        move_to_return = None
        for move in moves:
            ns = self.next_state(board, curr, move[0], move[1])
            score, _ = self._minmax(ns, 
                                    player,
                                    opp,
                                    curr=opp if curr == player else player,
                                    depth= depth - 1,
                                    alpha=alpha, 
                                    beta=beta)

            # alpha beta pruning
            if curr == player:
                if score > alpha:
                    alpha = score
                    move_to_return = move
                if alpha >= beta:
                    break
            else:
                if score < beta:
                    beta = score
                if alpha >= beta:
                    break
    
        # return either max or min, respectively
        if curr == player:
            return (alpha, move_to_return)
        else:
            return (beta, None)

def print_board(board):
    for i in range(len(board[0])):
        row = [board[j][i] for j in range(len(board))]
        row = map(str, row)
        print(" | ".join(row))

In [2]:
import copy

blank = "-"

# an implementation for tictactoe AI using minmax and alphabeta pruning
# board array access is board[col][row]

# boards are always squares
def make_board(width):
    return [[blank] * width for _ in range(width)]
              
def possible_moves(board, curr):  
    d = len(board)
    return [(y,x) for y in range(d) for x in range(d) if board[y][x] == blank]

# create a new board but with position (col, row) changed to player
def next_state(old, player, col,row):
    new = copy.deepcopy(old)
    new[col][row] = player
    return new

def get_winner(board):
    # look for vertical win
    for col in board:
        if len(set(col)) == 1 and col[0] != blank:
            return col[0]
            
    # horizontal win
    d = len(board)
    rows = [[board[i][x] for i in range(d)] for x in range(d)]
    for row in rows:
        if len(set(row)) == 1 and row[0] != blank:
            return row[0]
            
    # diagonal win: top left to bottom right
    tl_br_diag = [col[i] for i,col in enumerate(board)]
    if len(set(tl_br_diag)) == 1 and tl_br_diag[0] != blank:
        return tl_br_diag[0]

    # bottom left to top right
    bl_tr_diag = [col[-i - 1] for i,col in enumerate(board)]
    if len(set(bl_tr_diag)) == 1 and bl_tr_diag[0] != blank:
        return bl_tr_diag[0]
    
    # no winner
    return None

def game_over(board, player, opp):
    if get_winner(board) is not None:
        return True

    for col in board:
        for cell in col:
            if cell == blank:
                return False
    return True

# returns 10 if you are the winner, -10 if you are the loser and 0 otherwise (tie, or game not over)
def evaluate(board, you, _):
    win = get_winner(board)
    if win is None:
        return 0
    return 10 if win == you else -10
                
def play():
    board = make_board(3)
    player = "X"
    opp = "O"

    ai = AI(ai_piece=opp,
            opp=player,
            depth=9,
            game_over_fun=game_over,
            eval_fun=evaluate,
            moves_fun=possible_moves,
            next_state_fun=next_state)
    
    print("You are X")
    print("Enter your moves as: col row")

    while(True):
        print("Your Turn: ")
        print_board(board)
        var = input()
        y, x = map(int, var.split())
        if board[y][x] != blank:
            print("Invalid move!")
            continue        
    
        board = next_state(board, player, y,x)
        print_board(board)
        winner = get_winner(board)
        if game_over(board, player, opp):
            if winner != None:
                if winner == player:
                    print("You win!")
                else:
                    print("You lose!")
            else:
                print("Draw!")
            break

        print("Their turn...")
    
        score, ai_move = ai.get_move(board)
        print(ai_move)
        board = next_state(board, opp, ai_move[0], ai_move[1])
        print_board(board)
        winner = get_winner(board)    
        if game_over(board, player, opp):
            if winner != None:
                if winner == player:
                    print("You win!")
                else:
                    print("You lose!")
            else:
                print("Draw!")
            break

In [3]:
play()

You are X
Enter your moves as: col row
Your Turn: 
- | - | -
- | - | -
- | - | -
1 1
- | - | -
- | X | -
- | - | -
Their turn...
(0, 0)
O | - | -
- | X | -
- | - | -
Your Turn: 
O | - | -
- | X | -
- | - | -
2 0
O | - | X
- | X | -
- | - | -
Their turn...
(0, 2)
O | - | X
- | X | -
O | - | -
Your Turn: 
O | - | X
- | X | -
O | - | -
0 1
O | - | X
X | X | -
O | - | -
Their turn...
(2, 1)
O | - | X
X | X | O
O | - | -
Your Turn: 
O | - | X
X | X | O
O | - | -
1 2
O | - | X
X | X | O
O | X | -
Their turn...
(1, 0)
O | O | X
X | X | O
O | X | -
Your Turn: 
O | O | X
X | X | O
O | X | -
2 2
O | O | X
X | X | O
O | X | X
Draw!
